## Data Preparation Pipeline

##### Downloading the Dataset from Hugging Face

In [1]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

train_texts= dataset["train"]["text"]
test_texts = dataset["test"]["text"]

##### Tokenization


In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")


dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        truncation=True, 
        padding="max_length",
        max_length=128         
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

##### Preparing for PyTorch

In [3]:
from torch.utils.data import DataLoader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

def add_labels(examples):
    examples["labels"] = examples["input_ids"].clone()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)




Map:   0%|          | 0/36718 [00:00<?, ? examples/s]